# Preparation

In [53]:
!pip install spacy --user
!pip install import_ipynb --user
!pip install torchvision --user
!pip install torchtext --user

**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke P

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
  Using cached torch-1.10.2-cp36-cp36m-manylinux1_x86_64.whl (881.9 MB)
  Attempting uninstall: torch
    Found existing installation: torch 1.10.1
    Uninstalling torch-1.10.1:
      Successfully uninstalled torch-1.10.1
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.11.2 requires torch==1.10.1, but you have torch 1.10.2 which is incompatible.


In [54]:
!pip install --upgrade pip --user
!pip install setuptools-rust --user

**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.
**** Missing build for a loaded module for Python version "3.6.9". 
Directory does not exist: /share/pkg.7/pytorch/1.10.2/install/lib/SCC/../python3.6/site-packages

Please see https://github.com/pypa/pip/issues/5599 for advice on fixing the underlying issue.
To avoid this problem you can invoke Python with '-m pip' instead of running pip directly.


In [82]:
from spacy.cli.download import download

download(model="en_core_web_sm")
download(model="de_core_news_sm")


✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


In [78]:
import pickle
import torch

import torch.nn as nn
import torch.optim as optim
import spacy

import math
import time

import Transformer
from TransformerDataLoader import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [79]:
save_dir = './wmt17_en_de_processed.pkl'
with open(save_dir, 'rb') as f:
    data = pickle.load(f)

train_dataset, valid_dataset, test_dataset = data['train'],data['valid'],data['test']

src_vocab, trg_vocab = data['vocab']['src'], data['vocab']['trg']

In [80]:
print(f"training dataset: {len(train_dataset)}")
print(f"validation dataset: {len(valid_dataset)}")
print(f"testing dataset: {len(test_dataset)}")

training dataset: 4590101
validation dataset: 3000
testing dataset: 3003


In [81]:
train_dataset[0][0]

'resumption of the session'

In [82]:
spacy_en = spacy.load('en_core_web_sm')
spacy_de = spacy.load('de_core_news_sm')

def tokenize_de(text):
    return [token.text for token in spacy_de.tokenizer(text)]

def tokenize_en(text):
    return [token.text for token in spacy_en.tokenizer(text)]

# Check the batch data

In [70]:
BATCH_SIZE = 1
data_path = "./wmt17_en_de_processed.pkl"

train_iterator = TransformerDataLoader(data_path, 'train', batch_size=BATCH_SIZE)
valid_iterator = TransformerDataLoader(data_path, 'valid', batch_size=BATCH_SIZE)
test_iterator = TransformerDataLoader(data_path, 'test', batch_size=BATCH_SIZE)

In [71]:
# check for first batch
for i, batch in enumerate(train_iterator):
    src, trg = next(iter(train_iterator))

    print(f"first batch size: {src.shape}")
    print(f"first batch size: {trg.shape}")

    for i in range(src.shape[1]):
        print(f"{i}: {src[0][i].item()}") # the first src sentence

    break

first batch size: torch.Size([1, 26])
first batch size: torch.Size([1, 27])
0: 2
1: 581434
2: 130268
3: 83673
4: 4983
5: 450410
6: 158
7: 19
8: 115116
9: 144162
10: 13246
11: 10051
12: 142067
13: 105496
14: 140930
15: 19
16: 68889
17: 69062
18: 36145
19: 663371
20: 669
21: 823440
22: 115116
23: 1392282
24: 39
25: 3


# Train the model

In [83]:
dim_model = 256
dim_hidden = 512
dim_vocab = len(trg_vocab)
num_layers = 3
num_heads = 8

In [84]:
torch.cuda.empty_cache()

In [85]:
model = Transformer.TransformerModel(dim_model, dim_hidden, dim_vocab, N=num_layers, h=num_heads).to(device)

RuntimeError: CUDA out of memory. Tried to allocate 2.17 GiB (GPU 0; 15.78 GiB total capacity; 13.06 GiB already allocated; 1.38 GiB free; 13.06 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model)} trainable parameters')

In [ ]:
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

In [ ]:
# to index(with padding)
from helper import *

# Adam optimizer
LEARNING_RATE = 0.0005
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# ignore the padding index
criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

In [ ]:
# train function
def train(model, iterator, optimizer, criterion, clip=1):
    model.train() # set as train model
    epoch_loss = 0

    # iterate through whole data set
    for i, batch in enumerate(iterator):
        src, trg = next(iter(iterator))
        src.to(device)
        trg.to(device)

        optimizer.zero_grad()

        # exclude the end of the sentence
        # start with the beginning of the sentence
        src_padding_mask = Transformer.get_padding_mask(src, PAD_IDX)
        trg_padding_mask = Transformer.get_padding_mask(trg, PAD_IDX)
        peek_mask = Transformer.get_peek_mask(trg)
        output = model(src, trg,src_padding_mask,trg_padding_mask,peek_mask)

        # output: [batch size, trg_len - 1, output_dim]
        # trg: [batch size, trg_len]
        output_dim = output.shape[-1]

        output = output.contiguous().view(-1, output_dim)
        # Exclude index 0 (<sos>) of output word
        trg = trg.contiguous().view(-1)

        # output: [batch size * trg_len - 1, output_dim]
        # trg: [batch size * trg len - 1]

        print(output.shape)
        print(trg.shape)
        
        # calculate the loss
        loss = criterion(output, trg)
        loss.backward()

        # gradient clipping
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        # update the parameters
        optimizer.step()

        # calculate total loss
        epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
# evaluation
def evaluate(model, iterator, criterion):
    model.eval() # change to evaluation mode
    epoch_loss = 0

    with torch.no_grad():
        # Checking the entire evaluation data
        for i, batch in enumerate(iterator):
            src, trg = next(iter(iterator))
            src.to(device)
            trg.to(device)

            # exclude the end of the sentence
            # start with the beginning of the sentence
            src_padding_mask = Transformer.get_padding_mask(src, PAD_IDX)
            trg_padding_mask = Transformer.get_padding_mask(trg, PAD_IDX)
            peek_mask = Transformer.get_peek_mask(trg)
            output = model(src, trg[:,:-1],src_padding_mask,trg_padding_mask,peek_mask)

            # output: [batch size, trg_len - 1, output_dim]
            # trg: [batch size, trg_len]
            output_dim = output.shape[-1]

            output = output.contiguous().view(-1, output_dim)
            # Exclude index 0 (<sos>) of output word
            trg = trg[:,1:].contiguous().view(-1)

            # output: [batch size * trg_len - 1, output_dim]
            # trg: [batch size * trg len - 1]

            # calculate the loss
            loss = criterion(output, trg)

            # calculate total loss
            epoch_loss += loss.item()

    return epoch_loss / len(iterator)

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

main process

In [ ]:
N_EPOCHS = 10
CLIP = 1
best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    start_time = time.time() # record the start time

    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)

    end_time = time.time() # record the end time
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'transformer.pt')

    print(f'Epoch: {epoch + 1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} ')
    print(f'\tValidation Loss: {valid_loss:.3f}')

In [ ]:
# test set

model.load_state_dict(torch.load('transformer_german_to_english.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'Test Loss: {test_loss:.3f}')

# Visualization

In [ ]:
# translation
def translate_sentence(sentence, src_field, trg_field, model, device, max_len=50, logging=True):
    model.eval() # change into the evaluation mode

    if isinstance(sentence, str):
        nlp = spacy.load('de')
        tokens = [token.text.lower() for token in nlp(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # append <sos> token at the beginning and <eos> token at the end
    tokens = [src_field.init_token] + tokens + [src_field.eos_token]
    if logging:
        print(f"full source token: {tokens}")

    src_indexes = [src_field.vocab.stoi[token] for token in tokens]
    if logging:
        print(f"source sentence index: {src_indexes}")

    src_tensor = torch.LongTensor(src_indexes).unsqueeze(0).to(device)

    # Create a mask according to the source statement
    src_mask = model.make_src_mask(src_tensor)

    # Get the output value by putting the source sentence in the encoder
    with torch.no_grad():
        enc_src = model.Encoder(src_tensor)

    # Make sure you have only one <sos> token at first
    trg_indexes = [trg_field.vocab.stoi[trg_field.init_token]]

    for i in range(max_len):
        trg_tensor = torch.LongTensor(trg_indexes).unsqueeze(0).to(device)

        # Create a mask according to the output sentence
        trg_mask = model.make_trg_mask(trg_tensor)

        with torch.no_grad():
            output = model.Decoder((trg_tensor, enc_src))

        pred_token = output.argmax(2)[:,-1].item()
        trg_indexes.append(pred_token) # add to output statement

        # The moment you meet <eos>, it ends
        if pred_token == trg_field.vocab.stoi[trg_field.eos_token]:
            break

    #  Convert each output word index to an actual word
    trg_tokens = [trg_field.vocab.itos[i] for i in trg_indexes]

    # Returns the output statement excluding the first <sos>
    return trg_tokens[1:]